# Complément : Charger et compresser une vraie image

**Niveau :** Première NSI

Ce notebook complète le TP principal en montrant comment appliquer la transformation de Haar sur de **vraies images**.

## Méthodes pour charger une image

Il existe plusieurs façons de charger une image en Python :
1. Avec **matplotlib** (simple)
2. Avec **PIL/Pillow** (plus de contrôle)
3. Avec **imageio** (moderne)
4. Créer une image à partir d'une URL

## Importation des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as mpimg

# Pour charger des images depuis internet (optionnel)
try:
    from PIL import Image
    import requests
    from io import BytesIO
    PIL_DISPONIBLE = True
except ImportError:
    PIL_DISPONIBLE = False
    print("PIL n'est pas installé. Utilisez : pip install pillow requests")

## Rappel : Fonctions du TP principal

On réimporte les fonctions nécessaires du TP.

In [ ]:
def transformation_haar_2x2(carre):
    """Transformation de Haar sur un carré 2x2."""
    a, b = carre[0, 0], carre[0, 1]
    c, d = carre[1, 0], carre[1, 1]
    
    rouge = (a + b + c + d) / 4
    bleu = ((b - a) + (d - c)) / 4
    vert = ((c - a) + (d - b)) / 4
    violet = ((b + c) - (a + d)) / 4
    
    return np.array([[rouge, bleu], [vert, violet]])

def transformation_haar_image(image):
    """Transformation de Haar sur une image complète."""
    hauteur, largeur = image.shape
    nouvelle_hauteur = hauteur // 2
    nouvelle_largeur = largeur // 2
    
    rouge = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    bleu = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    vert = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    violet = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    
    for i in range(0, hauteur, 2):
        for j in range(0, largeur, 2):
            bloc = image[i:i+2, j:j+2]
            transforme = transformation_haar_2x2(bloc)
            
            rouge[i//2, j//2] = transforme[0, 0]
            bleu[i//2, j//2] = transforme[0, 1]
            vert[i//2, j//2] = transforme[1, 0]
            violet[i//2, j//2] = transforme[1, 1]
    
    resultat = np.zeros_like(image)
    resultat[0:nouvelle_hauteur, 0:nouvelle_largeur] = rouge
    resultat[0:nouvelle_hauteur, nouvelle_largeur:largeur] = bleu
    resultat[nouvelle_hauteur:hauteur, 0:nouvelle_largeur] = vert
    resultat[nouvelle_hauteur:hauteur, nouvelle_largeur:largeur] = violet
    
    return resultat, rouge, bleu, vert, violet

def transformation_haar_multi_niveaux(image, niveaux=3):
    """Transformation de Haar à plusieurs niveaux."""
    resultat = image.copy()
    hauteur, largeur = image.shape
    
    for niveau in range(niveaux):
        h = hauteur // (2 ** niveau)
        l = largeur // (2 ** niveau)
        
        if h < 2 or l < 2:
            break
            
        quadrant = resultat[0:h, 0:l]
        transforme, _, _, _, _ = transformation_haar_image(quadrant)
        resultat[0:h, 0:l] = transforme
    
    return resultat

def compresser(image_transformee, seuil=10):
    """Compression par seuillage."""
    compresse = image_transformee.copy()
    masque = np.abs(compresse) < seuil
    compresse[masque] = 0
    return compresse

print("✓ Fonctions chargées avec succès")

## Méthode 1 : Charger une image depuis un fichier local

### Avec matplotlib (le plus simple)

In [ ]:
# IMPORTANT: Remplacez 'mon_image.jpg' par le chemin de votre image
# Exemples de chemins:
# - Même dossier que le notebook: 'photo.jpg'
# - Sous-dossier: 'images/photo.jpg'
# - Chemin absolu: '/home/utilisateur/Images/photo.jpg'

# Pour tester, on va d'abord créer une image de test
# Dans un vrai TP, les élèves chargeraient leur propre image

def creer_image_test(nom_fichier='test_image.png'):
    """Crée une image de test pour la démonstration."""
    # Créer une image simple avec des formes
    img = np.ones((256, 256)) * 200  # Fond gris clair
    
    # Ajouter un rectangle noir
    img[50:150, 50:150] = 50
    
    # Ajouter un rectangle blanc
    img[100:200, 120:220] = 250
    
    # Ajouter un dégradé
    for i in range(256):
        img[i, i] = i
    
    # Sauvegarder l'image
    plt.imsave(nom_fichier, img, cmap='gray', vmin=0, vmax=255)
    return nom_fichier

# Créer l'image de test
fichier_test = creer_image_test()
print(f"Image de test créée : {fichier_test}")

In [ ]:
# Charger l'image avec matplotlib
image_couleur = mpimg.imread(fichier_test)

print(f"Type : {type(image_couleur)}")
print(f"Forme : {image_couleur.shape}")
print(f"Type de données : {image_couleur.dtype}")
print(f"Valeurs : min={image_couleur.min():.3f}, max={image_couleur.max():.3f}")

# Afficher l'image
plt.figure(figsize=(8, 8))
plt.imshow(image_couleur, cmap='gray')
plt.title("Image chargée")
plt.colorbar()
plt.show()

## Conversion en niveaux de gris

Pour appliquer notre transformation de Haar, l'image doit être en **niveaux de gris** (2D) et non en couleur (3D).

In [ ]:
def convertir_en_gris(image):
    """
    Convertit une image en niveaux de gris.
    
    - Si l'image est déjà en niveaux de gris (2D), la retourne telle quelle
    - Si l'image est en couleur (3D avec RGB ou RGBA), la convertit
    """
    if len(image.shape) == 2:
        # Déjà en niveaux de gris
        img_gris = image
    elif len(image.shape) == 3:
        if image.shape[2] == 3:  # RGB
            # Formule standard : Y = 0.299*R + 0.587*G + 0.114*B
            img_gris = 0.299 * image[:,:,0] + 0.587 * image[:,:,1] + 0.114 * image[:,:,2]
        elif image.shape[2] == 4:  # RGBA
            # Ignorer le canal alpha
            img_gris = 0.299 * image[:,:,0] + 0.587 * image[:,:,1] + 0.114 * image[:,:,2]
        else:
            raise ValueError(f"Format d'image non supporté : {image.shape}")
    else:
        raise ValueError(f"Dimension d'image non supportée : {image.shape}")
    
    # S'assurer que les valeurs sont dans [0, 255]
    if img_gris.max() <= 1.0:
        img_gris = img_gris * 255
    
    return img_gris

# Convertir l'image
image_gris = convertir_en_gris(image_couleur)

print(f"Forme après conversion : {image_gris.shape}")
print(f"Valeurs : min={image_gris.min():.1f}, max={image_gris.max():.1f}")

plt.figure(figsize=(8, 8))
plt.imshow(image_gris, cmap='gray', vmin=0, vmax=255)
plt.title("Image en niveaux de gris")
plt.colorbar(label="Intensité (0-255)")
plt.show()

## Redimensionner l'image (si nécessaire)

La transformation de Haar fonctionne mieux quand les dimensions sont des **puissances de 2** (256, 512, 1024, etc.).

In [ ]:
def redimensionner_puissance_2(image, taille_max=512):
    """
    Redimensionne l'image pour avoir des dimensions en puissance de 2.
    """
    hauteur, largeur = image.shape
    
    # Trouver la plus grande puissance de 2 inférieure à taille_max
    puissances = [2**i for i in range(4, 12)]  # 16, 32, 64, 128, 256, 512, 1024, 2048
    taille = min([p for p in puissances if p <= min(hauteur, largeur, taille_max)])
    
    # Rogner l'image au centre
    debut_h = (hauteur - taille) // 2
    debut_l = (largeur - taille) // 2
    
    image_redim = image[debut_h:debut_h+taille, debut_l:debut_l+taille]
    
    return image_redim

# Redimensionner
image_finale = redimensionner_puissance_2(image_gris, taille_max=256)

print(f"Taille originale : {image_gris.shape}")
print(f"Taille finale : {image_finale.shape}")

plt.figure(figsize=(8, 8))
plt.imshow(image_finale, cmap='gray', vmin=0, vmax=255)
plt.title(f"Image {image_finale.shape[0]}×{image_finale.shape[1]}")
plt.colorbar()
plt.show()

## Application de la transformation de Haar

Maintenant qu'on a une image en niveaux de gris avec des dimensions en puissance de 2, on peut appliquer la transformation !

In [ ]:
# Calculer le nombre de niveaux possibles
taille = image_finale.shape[0]
niveaux_max = int(np.log2(taille))
niveaux = min(6, niveaux_max)  # Limiter à 6 niveaux

print(f"Nombre de niveaux de transformation : {niveaux}")

# Appliquer la transformation
image_transformee = transformation_haar_multi_niveaux(image_finale, niveaux=niveaux)

# Visualiser
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(image_finale, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Image originale")
axes[0].axis('off')

axes[1].imshow(image_transformee, cmap='gray')
axes[1].set_title(f"Transformation de Haar ({niveaux} niveaux)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Visualisation détaillée des quadrants

Regardons les différentes composantes après la première itération.

In [ ]:
# Première itération seulement
_, rouge, bleu, vert, violet = transformation_haar_image(image_finale)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Image originale
axes[0, 0].imshow(image_finale, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title("Image originale", fontsize=14)
axes[0, 0].axis('off')

# Rouge (moyennes = structure)
axes[0, 1].imshow(rouge, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title("🔴 Moyennes (structure générale)", fontsize=14)
axes[0, 1].axis('off')

# Placeholder
axes[0, 2].axis('off')

# Bleu (contours verticaux)
axes[1, 0].imshow(np.abs(bleu), cmap='hot')
axes[1, 0].set_title("🔵 Différences verticales", fontsize=14)
axes[1, 0].axis('off')

# Vert (contours horizontaux)
axes[1, 1].imshow(np.abs(vert), cmap='hot')
axes[1, 1].set_title("🟢 Différences horizontales", fontsize=14)
axes[1, 1].axis('off')

# Violet (contours obliques)
axes[1, 2].imshow(np.abs(violet), cmap='hot')
axes[1, 2].set_title("🟣 Différences obliques", fontsize=14)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("Observation :")
print("- Le carré rouge contient la structure générale de l'image")
print("- Les carrés bleu, vert et violet mettent en évidence les contours")
print("- Les zones sombres = peu de changement (compressibles !)")
print("- Les zones claires = changements importants (détails à conserver)")

## Test de compression avec différents seuils

In [ ]:
# Tester différents niveaux de compression
seuils = [5, 15, 30, 50]

fig, axes = plt.subplots(1, len(seuils)+1, figsize=(20, 4))

# Image transformée originale
axes[0].imshow(image_transformee, cmap='gray')
axes[0].set_title("Transformée\n(sans compression)", fontsize=12)
axes[0].axis('off')

# Compressions avec différents seuils
for i, seuil in enumerate(seuils):
    img_compresse = compresser(image_transformee, seuil)
    nb_zeros = np.sum(img_compresse == 0)
    pourcentage = (nb_zeros / img_compresse.size) * 100
    
    axes[i+1].imshow(img_compresse, cmap='gray')
    axes[i+1].set_title(f"Seuil = {seuil}\n{pourcentage:.1f}% de zéros", fontsize=12)
    axes[i+1].axis('off')

plt.suptitle("Compression avec différents seuils", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Plus le seuil est élevé, plus on compresse (plus de zéros)")
print("   Mais attention : on perd aussi plus de détails !")

## Méthode 2 : Charger une image depuis une URL (bonus)

Si PIL est installé, on peut charger une image directement depuis Internet !

In [ ]:
if PIL_DISPONIBLE:
    def charger_image_url(url):
        """
        Charge une image depuis une URL.
        """
        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content))
            # Convertir en array numpy
            img_array = np.array(img)
            return img_array
        except Exception as e:
            print(f"Erreur lors du chargement : {e}")
            return None
    
    # Exemple avec une image de test (licence libre)
    # URL d'exemple - remplacer par une image de votre choix
    print("Pour charger une image depuis internet :")
    print("url = 'https://exemple.com/image.jpg'")
    print("image = charger_image_url(url)")
else:
    print("PIL n'est pas disponible.")
    print("Pour installer : pip install pillow requests")

## 🎯 Exercice : Analysez votre propre image !

### Instructions :

1. **Placez une image** dans le même dossier que ce notebook
2. **Modifiez** le nom du fichier dans la cellule ci-dessous
3. **Exécutez** toutes les cellules pour voir la compression

### Questions à explorer :

- Quelle est la différence entre une photo et un dessin simple ?
- Les zones uniformes se compressent-elles mieux ?
- Quel seuil donne le meilleur compromis qualité/compression ?

In [ ]:
# TODO: Remplacez par le nom de votre image
MON_IMAGE = 'mon_image.jpg'  # ← Modifiez ici !

try:
    # Charger l'image
    mon_img = mpimg.imread(MON_IMAGE)
    
    # Convertir en gris
    mon_img_gris = convertir_en_gris(mon_img)
    
    # Redimensionner
    mon_img_finale = redimensionner_puissance_2(mon_img_gris, taille_max=256)
    
    # Transformer
    niveaux = min(6, int(np.log2(mon_img_finale.shape[0])))
    mon_img_trans = transformation_haar_multi_niveaux(mon_img_finale, niveaux=niveaux)
    
    # Afficher
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    axes[0].imshow(mon_img_finale, cmap='gray', vmin=0, vmax=255)
    axes[0].set_title("Votre image", fontsize=16)
    axes[0].axis('off')
    
    axes[1].imshow(mon_img_trans, cmap='gray')
    axes[1].set_title("Transformée de Haar", fontsize=16)
    axes[1].axis('off')
    
    plt.show()
    
    print("✓ Image chargée et transformée avec succès !")
    
except FileNotFoundError:
    print(f"❌ Fichier '{MON_IMAGE}' introuvable !")
    print("Vérifiez que l'image est dans le même dossier que le notebook.")
except Exception as e:
    print(f"❌ Erreur : {e}")

## 📊 Statistiques de compression

Analysons en détail le taux de compression.

In [ ]:
# Analyser la distribution des coefficients
coeffs = image_transformee.flatten()

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogramme des coefficients
axes[0].hist(coeffs, bins=100, edgecolor='black')
axes[0].set_xlabel("Valeur du coefficient")
axes[0].set_ylabel("Fréquence")
axes[0].set_title("Distribution des coefficients de Haar")
axes[0].grid(True, alpha=0.3)

# Courbe de compression
seuils_test = range(0, 101, 2)
taux = []

for s in seuils_test:
    img_comp = compresser(image_transformee, s)
    pct = (np.sum(img_comp == 0) / img_comp.size) * 100
    taux.append(pct)

axes[1].plot(seuils_test, taux, 'b-', linewidth=2)
axes[1].set_xlabel("Seuil de compression")
axes[1].set_ylabel("Pourcentage de coefficients à zéro (%)")
axes[1].set_title("Courbe de compression")
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, 100)
axes[1].set_ylim(0, 100)

plt.tight_layout()
plt.show()

# Statistiques
print("\n📈 Statistiques des coefficients :")
print(f"  Moyenne : {np.mean(coeffs):.2f}")
print(f"  Écart-type : {np.std(coeffs):.2f}")
print(f"  Min : {np.min(coeffs):.2f}")
print(f"  Max : {np.max(coeffs):.2f}")
print(f"\n  Coefficients < 10 : {(np.abs(coeffs) < 10).sum() / len(coeffs) * 100:.1f}%")
print(f"  Coefficients < 30 : {(np.abs(coeffs) < 30).sum() / len(coeffs) * 100:.1f}%")
print(f"  Coefficients < 50 : {(np.abs(coeffs) < 50).sum() / len(coeffs) * 100:.1f}%")

## 🚀 Pour aller plus loin

### Idées de projets :

1. **Comparer différentes images** : photo vs dessin vs texte
2. **Créer une galerie** d'images compressées à différents niveaux
3. **Mesurer le temps** de calcul selon la taille de l'image
4. **Implémenter la reconstruction** et mesurer l'erreur
5. **Comparer avec JPEG classique** (si PIL disponible)

### Ressources :

- Documentation matplotlib : https://matplotlib.org/
- Documentation PIL : https://pillow.readthedocs.io/
- Images libres de droits : Unsplash, Pixabay, Pexels

---

**Complément au TP JPEG 2000 pour 1ère NSI**